# Reporte de avances modelo Sentan-I

In [1]:
from Src.pre_processing import load_data, split, to_tensor, to_labels 
from Src.Sentan_Model import Sentan_Model
from Src.Dias_Model import Dias_Model

import torch
from sklearn.model_selection import train_test_split
from torch.optim import Adam
from torch import nn

from torchmetrics import F1Score, Recall, Precision, Accuracy
import wandb


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### Ciclo de entrenamiento

In [2]:

def train(model, X_train, y_train, X_test, y_test, device, optimizer, lossFn, epochs):
    f1 = F1Score(num_classes=8)
    recall = Recall(average='macro', num_classes=8)
    precision = Precision(average='macro', num_classes=8)
    accuracy = Accuracy()

    X_train, y_train = to_tensor(X_train, y_train)
    X_test, y_test = to_tensor(X_test, y_test)
    X_train, y_train = X_train.to(device), y_train.to(device)
    X_test, y_test = X_test.to(device), y_test.to(device)

    # Entrenamiento del modelo
    model.train()
    for epoch in range(0, epochs):

        optimizer.zero_grad()

        pred = model(X_train)

        loss = lossFn(pred, y_train)
        loss.backward()
        optimizer.step()

        train_accuracy = accuracy(pred, y_train)

        # Validacion
        with torch.no_grad():
            pred = model(X_test)

            # Login de resultados a Weights and biases
            wandb.log({'Train Accuracy': train_accuracy, 'Train Loss': loss, 'Validation Accuracy': accuracy(
                pred, y_test), 'F1 Score': f1(pred, y_test), 'Recall': recall(pred, y_test), 'Precision': precision(pred, y_test)})


### Preparación de los datos

Los datos serán cargados desde una versión del conjunto de datos RAVDESS, se representan como un conjunto de 193 variables por cada muestra.

In [3]:
data_path = 'Data/data.pkl'
X, y, labels = load_data(data_path)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

### Hiperparametros para pruebas

In [4]:
epochs = 10
learning_rate = 1e-3
batch_size = X_train.shape[0]


# Configuración de entorno weights and biases
wandb.config = {
    "model-name": 'Dias model',
    "learning_rate": learning_rate,
    "epochs": epochs,
    "batch_size": batch_size

}

In [5]:
# Entrenamiento de n modelos
models = [Dias_Model(), Dias_Model()]


# Model loop
for model in models:
    
    # Activación del entorno
    wandb.init(project="Clasificador de sentimientos")
    model = model.to(device)


    loss_fn = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr=learning_rate)


    # Entrenamiento del modelo
    train(model, X_train, y_train, X_test, y_test, device, optimizer, lossFn=loss_fn, epochs=epochs)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: solrak. Use `wandb login --relogin` to force relogin
